Поиск расхождений в данных сверок взаиморасчетом с контрагентом

In [17]:
import pandas as pd

# Готовим файл с версией сверки от поставщика

url1='https://docs.google.com/spreadsheets/d/18b4cT-FjoGVI2I9xcXr2dM6-AFV5GbRS/edit?usp=sharing&ouid=116045163522511725897&rtpof=true&sd=true'
url1='https://drive.google.com/uc?id=' + url1.split('/')[-2]

df_agent = pd.read_excel(url1)
# Убираем все лишнее
df_agent = df_agent.drop([0,1,2,3,4,5,6,7]).fillna('')
df_agent = df_agent.drop(['Unnamed: 0','Unnamed: 3','Unnamed: 3','Unnamed: 5','Unnamed: 7',
                          'Unnamed: 8','Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12',
                          'Unnamed: 13', 'Unnamed: 14'],
                           axis = 1)\
                    .rename(columns = {'Unnamed: 1' : 'date',
                                       'Unnamed: 2' : 'op',
                                       'Unnamed: 4' : '1',
                                       'Unnamed: 6' : '2'})
# Разделяем операции на оплату и отгрузку

t = []
for op in df_agent.op:
    if op.find('Платеж') != -1:
         t.append('payment')
    elif op.find('Реал') != -1:
          t.append('shipment')
    else:
         t.append(0)
df_agent['op'] = t

# Убираем пустые строки

index_names = df_agent[df_agent['op'] == 0].index
df_agent.drop(index_names, inplace = True)

# Формируем строку для будущего сравнения

t = []
t = df_agent['1'].astype(str) + df_agent['2'].astype(str)
df_agent['ops'] = t

df_agent['ops'] = df_agent.ops.astype(str) + ' ' + df_agent.op

# оставляем только нужное
df_agent = df_agent[['date', 'ops']]
df_agent

,date,ops
8,10.01.24,43260 shipment
9,10.01.24,33000 shipment
10,11.01.24,12880 payment
11,11.01.24,8920 payment
12,11.01.24,5505 payment
...,...,...
139,11.09.24,52360 shipment
140,18.09.24,67640 shipment
141,18.09.24,254 shipment
142,19.09.24,54750 payment


In [10]:
# Готовим файл с нашей версией сверки

url2='https://docs.google.com/spreadsheets/d/104t1JmAZY0BrFnXcrL1-3hmu87ZB2d-x/edit?usp=sharing&ouid=116045163522511725897&rtpof=true&sd=true'
url2='https://drive.google.com/uc?id=' + url2.split('/')[-2]

df_ours = pd.read_excel(url2)

df_ours = df_ours.drop([0,1,2,3,4,5,6,7,8,9,10,11], axis = 0)
df_ours = df_ours.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6',
                        'Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11','Unnamed: 12','Unnamed: 13'
                       ,'Unnamed: 14','Unnamed: 15'], axis = 1)\
                 .fillna('')\
                 .rename(columns = {'Акт сверки взаимных расчетов № 54 от 1 октября 2024 г.' : 'date',
                                    'Unnamed: 1':'op',
                                    'Unnamed: 7':'1'})

t = []
for op in df_ours.op:
    if op.find('Платеж') != -1:
         t.append('payment')
    elif op.find('Заку') != -1:
          t.append('shipment')
    else:
         t.append(0)
df_ours['op'] = t

index_names = df_ours[df_ours['op'] == 0].index
df_ours.drop(index_names, inplace = True)

df_ours.op =  df_ours['1'].astype(str) + ' ' + df_ours['op'].astype(str)

df_ours = df_ours.drop(['1'], axis = 1)

df_ours

,date,op
12,10.01.2024,43260 shipment
13,11.01.2024,33000 shipment
14,11.01.2024,8920 payment
15,11.01.2024,12880 payment
16,11.01.2024,5505 payment
...,...,...
142,16.09.2024,33000 shipment
143,18.09.2024,67640 shipment
144,19.09.2024,254 shipment
145,19.09.2024,54750 payment


In [11]:
df_lft = df_agent[-df_agent['ops'].isin(df_ours['op'])].reset_index(drop = True)
df_lft

,date,ops


In [12]:
df_rgt = df_ours[-df_ours['op'].isin(df_agent['ops'])].reset_index(drop = True)
df_rgt

,date,op


In [14]:
df_agent.groupby('ops').ops.value_counts().sort_values().tail(10)

ops
79960 shipment    1
8860 shipment     1
74880 payment     2
3040 payment      2
3280 shipment     2
3040 shipment     2
3720 shipment     2
3720 payment      3
33000 shipment    5
33000 payment     5
Name: count, dtype: int64

In [15]:
df_ours.groupby('op').op.value_counts().sort_values().tail(10)

op
79960 payment     1
8920 payment      1
8860 shipment     1
3040 payment      2
3280 shipment     2
3040 shipment     2
3720 shipment     2
3720 payment      3
33000 shipment    5
33000 payment     5
Name: count, dtype: int64

In [16]:
# Сравниваем списки операций по суммам, проходившем более 1 раза и видив расхождения в количестве операций с одной суммой 74880 и 3720 - признак того что операция задвоена, то етсь проведена дважды